In [ ]:
import pandas as pd
import numpy as np 
!pip install fuzzywuzzy
!pip install python-Levenshtein
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
!pip install swifter
import swifter

In [ ]:
def similiar(col):
    uniq = list(col.value_counts().index)[::-1]          # listing all unique levels in increasing order of frequency 
    uniq_values = list(col.value_counts())[::-1]         # listing count of all unique level in increasing order
    idx = (np.abs(np.asarray(uniq_values) - col.count()*0.05)).argmin()  
    # index of level which have frequency of about 5%, as a spelling mistake is unlikely to have more than 5% 
    new_dict={}          # dictionary for storing the mapping of incorrect spelling as key and correct value as value

    for i in range(idx):
        l = uniq[i+1:][::-1]      # for faster search we start for levels having largest frequency, hence reversed list
        for j in l:
            if fuzz.token_sort_ratio(uniq[i], j)> 67 :       # token sort ratio gives similarity coefficient in range(0-100)
                new_dict[uniq[i]]=j
                break

    return col.map(new_dict).fillna(col)


obj_cols = list(df.dtypes[df.dtypes == np.object].index) 
col_uni = df[obj_cols].nunique()/df[obj_cols].count()
df[list(col_uni[0.15>col_uni].index)] = df[list(col_uni[0.15>col_uni].index)].swifter.apply(similiar)

'\nobj_cols = list(df.dtypes[df.dtypes == np.object].index) \ncol_uni = df[obj_cols].nunique()/df[obj_cols].count()\ndf[list(col_uni[0.15>col_uni].index)] = df[list(col_uni[0.15>col_uni].index)].swifter.apply(similiar)'